<a href="https://colab.research.google.com/github/dlsun/Stat350F19/blob/master/Gaussian_Processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import Symbulate
!pip install -q symbulate
from symbulate import *

# Gaussian Processes

$\def\Cov{\text{Cov}}
\def\Var{\text{Var}}$

A random process $\{ X_t \}$ is called a **Gaussian process** if:
- $X_t$ has a normal (Gaussian) distribution for every $t$.
- $(X_s, X_t)$ is bivariate normal for every $s$ and $t$.
- $(X_{t_1}, ..., X_{t_n})$ is multivariate normal for any times $t_1, ..., t_n$.

Are Gaussian processes continuous-time or discrete-time? Are they continuous-valued or discrete-valued?

Gaussian processes are always continuous-valued, but they can be either 
continuous-time or discrete-time.

A Gaussian process is uniquely determined by its mean function $\mu(t)$ and autocovariance function $C(s, t)$.

## Examples of Gaussian Processes

\begin{align}
\mu(t) &= 0 & C(s, t) &= 2 e^{-(s-t)^2 / 2}
\end{align}

In [0]:
def mean(t):
  return 0

def cov(s, t):
  return 2 * exp(-(s - t) ** 2 / 2)

X = GaussianProcess(mean_func=mean, cov_func=cov)
X.draw().plot()
X.draw().plot()
X.draw().plot()

\begin{align}
\mu(t) &= 0 & C(s, t) &= 2 e^{-|s - t| / 2}
\end{align}

In [0]:
def mean(t):
  return 0

def cov(s, t):
  return 2 * exp(-abs(s - t) / 2)

X = GaussianProcess(mean_func=mean, cov_func=cov)
X.draw().plot()
X.draw().plot()
X.draw().plot()

\begin{align}
\mu(t) &= 0 & C(s, t) &= 0.2 s t
\end{align}

In [0]:
def mean(t):
  return 0

def cov(s, t):
  return 0.2 * s * t

X = GaussianProcess(mean_func=mean, cov_func=cov)
X.draw().plot()
X.draw().plot()
X.draw().plot()

\begin{align}
\mu(t) &= 0 & C(s, t) &= 0.5 \min(s, t)
\end{align}

In [0]:
def mean(t):
  return 0

def cov(s, t):
  return 0.5 * min(s, t)

X = GaussianProcess(mean_func=mean, cov_func=cov)
X.draw().plot()
X.draw().plot()
X.draw().plot()

# Calculations with Gaussian Processes

Let $\{ X(t) \}$ be a _stationary_ Gaussian process with mean function $\mu_X(t) = -1$ and autocovariance function $C_X(\Delta t) = 4e^{-(\Delta t)^2 / 2}$.

## Question 1

What is $P(X(5) > 1)$?

Since $\{ X(t) \}$ is a Gaussian process, $X(5)$ has a Gaussian (a.k.a. normal) distribution. To calculate probabilities with a normal distribution, we need to know its mean and standard deviation.

\begin{align}
E[X(5)] &= \mu(5) = -1 \\
\Var[X(5)] &= \Cov[X(5), X(5)] \\
&= C(5 - 5) \\
&= 4 e^{-0^2 / 2} \\
&= 4 \\
SD[X(5)] &= \sqrt{4} = 2
\end{align}

So $X(5) \sim \text{Normal}(\mu=-1, \sigma=2)$. We can use Symbulate to calculate $P(X(5) > 1)$.

In [0]:
1 - Normal(mean=-1, sd=2).cdf(1)

Let's check this answer by simulation.

In [0]:
def mean(t):
  return -1

def cov(s, t):
  return 4 * exp(-(s - t) ** 2 / 2)

X = GaussianProcess(mean_func=mean, cov_func=cov)

x5 = X(5).sim(10000)
x5.plot()
(x5 > 1).mean()

## Question 2

What is $P(X(5) - X(3) > 1)$?

Since $\{ X(t) \}$ is a Gaussian process, $(X(5), X(3))$ has a bivariate normal distribution. Furthermore, $X(5) - X(3)$ is a linear combination of a bivariate normal, and we know that linear combinations of a bivariate normal are themselves normal. 

So to calculate the probability, we just need to know the mean and standard deviation of $X(5) - X(3)$.

\begin{align}
E[X(5) - X(3)] &= E[X(5)] - E[X(3)] = \mu(5) - \mu(3) = (-1) - (-1) = 0\\
\Var[X(5) - X(3)] &= \Cov[X(5) - X(3), X(5) - X(3)] \\
&= \Cov[X(5), X(5)] - 2 \cdot \Cov[X(5), X(3)] + \Cov[X(3), X(3)]  \\
&= C(5 - 5) - 2 \cdot C(5 - 3) + C(3 - 3) \\
&= 4 e^{-0^2 / 2} - 2 \cdot 4 e^{-2^2 / 2} + 4 e^{-0^2 / 2} \\
&= 8(1 - e^{-2}) \\
\text{SD}[X(5)] &= \sqrt{8(1 - e^{-2})}
\end{align}

So $X(5) - X(3) \sim \text{Normal}(\mu=0, \sigma=\sqrt{8(1 - e^{-2})})$. We can use Symbulate to calculate $P(X(5) - X(3) > 1)$.

In [0]:
1 - Normal(mean=0, sd=sqrt(8 * (1 - exp(-2)))).cdf(1)

Now let's check the answer by simulation. Note that `X` is defined in the previous simulation.

In [0]:
diff = (X(5) - X(3)).sim(10000)
diff.plot()
(diff > 1).mean()